In [1]:
import pandas as pd
import parser
import imp

pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100

imp.reload(parser)

<module 'parser' from '/usr/lib/python3.6/lib-dynload/parser.cpython-36m-x86_64-linux-gnu.so'>

In [2]:
dataset = pd.read_excel("np_ngo_classification.xlsx")
dataset.head(1)

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Ссылка на соц сеть,Вид деятельности 1,...,Вид деятельности 3,Вид деятельности 4,Вид деятельности 5,Вид деятельности 6,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12
0,Москва,"Межрегиональная общественная благотворительная организация ""Общество защиты прав потребителей и ...",5032173157,2007-07-12,"г Москва, Орехово-Борисово Южное р-н, МКАД 23-й км, д 6",0,0,"http://greenstartup.ru/, http:/9911850.ru/",https://princip1.livejournal.com/,3_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
dataset[dataset['ИНН'] == 6320266667]

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Ссылка на соц сеть,Вид деятельности 1,...,Вид деятельности 3,Вид деятельности 4,Вид деятельности 5,Вид деятельности 6,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12
52,Тольятти,"Благотворительный Фонд ""Преобразование""",6320266667,2014-03-26,"Самарская обл, г Тольятти, Автозаводский р-н, пр-кт Степана Разина, д 4, кв 48",1,1,http://bf-pr.ru/,https://vk.com/bfpreobrazovanie\n,1_2,...,1_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Drop empty rows
sites = dataset[~dataset["Ссылка на сайт"].isnull()]
sites.drop(columns=["Ссылка на соц сеть"], inplace=True)
social = dataset[~dataset["Ссылка на соц сеть"].isnull()]

# Merging social web-page to a web-site, adding new data samples 
social["Ссылка на сайт"] = social["Ссылка на соц сеть"]
social.drop(columns=["Ссылка на соц сеть"], inplace=True)
concat_dataset = pd.concat([sites, social], ignore_index=True) 

# Fixing mistakes
concat_dataset["Ссылка на сайт"].loc[62] = "http://арктика-надежда.рф" 
concat_dataset["Ссылка на сайт"].loc[154] = "http://9911850.ru/"

/home/niwuro/Environments/WorkEnv/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/niwuro/Environments/WorkEnv/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/niwuro/Environments/WorkEnv/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  se

In [4]:
def link_splitter(url_string):
    result = []
    urls = []
    
    for url in url_string.split(","):
        urls.append(url)
        
    for url in urls:
        for each in url.split("\n"):
            result.append(each)
            
    return result

concat_dataset["Ссылка на сайт"] = concat_dataset["Ссылка на сайт"].apply(link_splitter)
dataset.dropna(axis=0, subset=["Ссылка на сайт"], inplace=True)
concat_dataset["num_sites"] = concat_dataset["Ссылка на сайт"].apply(len)

In [7]:
# ! WARNING: This cell needs to be executed 3 times!
# TODO: 3 times-loop
max_len = len(concat_dataset)
for i in range(3):
    for index, row in concat_dataset.iterrows():
        if len(row["Ссылка на сайт"])>1:
            sites = row["Ссылка на сайт"]
            last_site = sites.pop(-1)
            concat_dataset["Ссылка на сайт"].loc[index] = sites
            row["Ссылка на сайт"] = [last_site]
            concat_dataset.loc[max_len] = row
            max_len += 1

In [25]:
concat_dataset[concat_dataset['ИНН'] == 6320266667]

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Вид деятельности 1,Вид деятельности 2,...,Вид деятельности 5,Вид деятельности 6,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12,num_sites,web_text
49,Тольятти,"Благотворительный Фонд ""Преобразование""",6320266667,2014-03-26,"Самарская обл, г Тольятти, Автозаводский р-н, пр-кт Степана Разина, д 4, кв 48",1,1,http://bf-pr.ru/,1_2,1_9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"[html, Благотворительный фонд ""Преобразование"" в Самаре - Помощь сиротам, инвалидам, [if IE 6]>\..."
118,Тольятти,"Благотворительный Фонд ""Преобразование""",6320266667,2014-03-26,"Самарская обл, г Тольятти, Автозаводский р-н, пр-кт Степана Разина, д 4, кв 48",1,1,https://vk.com/bfpreobrazovanie,1_2,1_9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"[xml version=""1.0"" encoding=""utf-8""?, html PUBLIC ""-//WAPFORUM//DTD XHTML Mobile 1.0//EN"" ""http:..."
178,Тольятти,"Благотворительный Фонд ""Преобразование""",6320266667,2014-03-26,"Самарская обл, г Тольятти, Автозаводский р-н, пр-кт Степана Разина, д 4, кв 48",1,1,None,1_2,1_9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,None


In [9]:
concat_dataset["Ссылка на сайт"] = concat_dataset["Ссылка на сайт"].apply(lambda lst: lst[0].strip())

In [10]:
# Adding http prefix 
def fix_url(url):
    if not isinstance(url, str):
        return None
    if len(url)<2:
        return None
    if not url.startswith("http"):
        return "http://"+url
    return url

concat_dataset["Ссылка на сайт"] = concat_dataset["Ссылка на сайт"].apply(fix_url)

In [12]:
concat_dataset

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Вид деятельности 1,Вид деятельности 2,...,Вид деятельности 4,Вид деятельности 5,Вид деятельности 6,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12,num_sites
0,Москва,"Межрегиональная общественная благотворительная организация ""Общество защиты прав потребителей и ...",5032173157,2007-07-12,"г Москва, Орехово-Борисово Южное р-н, МКАД 23-й км, д 6",0,0,http://greenstartup.ru/,3_2,6_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,Москва,"Межрегиональная благотворительная общественная организация инвалидов ""Общество пациентов с перви...",7702370774,2007-08-28,"г Москва, Мещанский р-н, ул Трифоновская, д 49, оф 41",1,1,http://oppid.ru/,2_1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Москва,"Некоммерческая организация Благотворительный фонд ""Столица Милосердия""",7702472455,2014-02-11,"г Москва, Мещанский р-н, ул Щепкина, д 9 стр 1",1,1,http://capital-mercy.ru,1_9,1_2,...,2_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Москва,"Российский общественный благотворительный фонд ветеранов (пенсионеров) войны, труда и Вооруженны...",7704058560,1998-06-05,"г Москва, р-н Соколиная Гора, ул Щербаковская, д 50/52",1,1,http://www.rbfondveteranov.ru/,1_4,5_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Москва,"Благотворительный фонд поддержки и реализации программ в сфере культуры ""АВС""",7704278848,2011-09-30,"г Москва, р-н Хамовники, пер Знаменский Б., д 13 стр 2",0,0,http://www.avccharity.com/,4_2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,Москва,"Благотворительный фонд ""Фонд Валерия Гергиева""",7709402778,2003-01-27,"г Москва, р-н Арбат, ул Молчановка Б., д 23 к 2",0,0,http://gergiev-charity.com/rus/,4_2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,Москва,Международный Благотворительный Фонд Владимира Спивакова,7710012370,1994-05-20,"г Москва, Пресненский р-н, Шведский тупик, д 3, оф 17",1,0,http://spivakov.ru/fund/contacts/,4_2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,Москва,"Благотворительный фонд поддержки детей с особенностями развития ""Я есть!""",7710479817,2012-05-02,"г Москва, Пресненский р-н, пер Кондратьевский Б., д 4 к 3, оф 29",1,1,http://yaest.ru/,1_7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,Москва,"Благотворительный фонд помощи бездомным животным ""Ковчег""",7716451494,2014-11-14,"г Москва, Ярославский р-н, Ярославское шоссе, д 2 к 1, кв 157",1,0,http://fond-kovcheg.ru,6_4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,Москва,Благотворительный фонд поддержки детей имени императрицы Александры Федоровны Романовой,7701064809,2011-03-22,"105082, ГОРОД МОСКВА, УЛИЦА ФРИДРИХА ЭНГЕЛЬСА, 75, СТР. 3, ОФ. № 1",1,1,http://www.vfonde.ru/,1_7,1_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [26]:
adress = 'http://tenfoundation.ru/'

In [ ]:
parser.get_text_contents('http://tenfoundation.ru')

In [15]:
concat_dataset["web_text"] = concat_dataset["Ссылка на сайт"].apply(parser.get_text_contents)

/home/niwuro/Projects/NCO/npo_classification/parser.py:34: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 34 of the file /home/niwuro/Projects/NCO/npo_classification/parser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


Error: HTTPConnectionPool(host='www.avccharity.com', port=80): Read timed out. (read timeout=27), url: http://www.avccharity.com/
Error: HTTPConnectionPool(host='delfin-center.ru', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f789079f320>: Failed to establish a new connection: [Errno -2] Name or service not known',)), url: http://delfin-center.ru/
Error: Invalid URL 'http:/9911850.ru/': No schema supplied. Perhaps you meant http://http:/9911850.ru/?, url: http:/9911850.ru/
Error: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?, url: None
Error: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?, url: None


In [13]:
concat_dataset.dropna(subset=["web_text"]).to_pickle(path="np_ngo_classification_links.pkl")

KeyError: ['web_text']

In [15]:
concat_dataset.to_pickle(path="np_ngo_classification_links.pkl")

In [18]:
concat_dataset['web_text'][1]

['html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"',
 'ОППИД | Общество пациентов с первичным иммунодефицитом',
 'a[href$=".pdf"], a[href$=".txt"], a[href$=".mp3"], a[href$=".aiff"], a[href$=".wav"], a[href$=".ogg"], a[href$=".wma"], a[href$=".zip"], a[href$=".rar"], a[href$=".ace"], a[href$=".tar.gz"], a[href$=".tgz"], a[href$=".mpg"], a[href$=".mpeg"], a[href$=".avi"], a[href$=".mv2"], a[href$=".mov"], a[href$=".mp4"], a[href$=".wmv"], a[href$=".swf"], a[href$=".xls"], a[href$=".doc"], a[href*="trackback"], a[href*="@"], a[href*="rss2"], a[href*="feed"], a[href*="postcomment"], a[href*="respond"], a[href*="action=edit"]\n{\n\t\tpadding-bottom:2px;\n\t\tpadding-left:22px;\n\t\tpadding-top:2px;\n\t\tbackground:transparent;\n\t}\n a[href$=".pdf"]\n\t\t{\n\t\t\tbackground: url(http://oppid.ru/wp-content/plugins/file-icons/icons/silkicons/page_white_acrobat.png) no-repeat scroll left center;\n\t\t}\n a[href$=".txt"]\n\t\t{\n\t\

In [14]:
concat_dataset["web_text"].loc[22]

KeyError: 'web_text'

In [20]:
mapping = pd.read_excel("np_ngo_mappings.xlsx")
mapping

,Социально-уязвимые группы населения,Unnamed: 1,1,Unnamed: 3
0,-,Проблемы людей с зависимостями,1_1,NaN
1,-,Бедность,1_2,NaN
2,-,Бездомность,1_3,NaN
3,-,Проблемы пожилых людей,1_4,NaN
4,-,"Проблемы заключенных и людей, освободившихся из МЛС",1_5,NaN
5,-,Проблемы людей с ВИЧ,1_6,NaN
6,-,Проблемы людей с инвалидностью,1_7,NaN
7,-,Проблемы мигрантов и беженцев,1_8,NaN
8,-,Социальное сиротство,1_9,NaN
9,-,Насилие,1_10,NaN


In [22]:
dataset["links"] = dataset["Ссылка на сайт"].apply(parser.get_links)

/home/niwuro/Projects/NCO/npo_classification/parser.py:16: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 16 of the file /home/niwuro/Projects/NCO/npo_classification/parser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


In [23]:
tr = cyrtranslit.to_cyrillic('http://oppid.ru/zasedanie-rabochey-gruppyi-po-redkim-zabolevaniyam-pod-predsedatelstvom-valentinyi-ivanovnyi-shirokovoy/', 'ru')

NameError: name 'cyrtranslit' is not defined

In [ ]:
nlp = Russian()
# nlp = spacy.load('xx_ent_wiki_sm')
tokenizer = Tokenizer(nlp.vocab)

In [ ]:
doc = nlp(tr)
print([token.text for token in doc])